<a href="https://colab.research.google.com/github/sznajder/ON-Minicurso-2022/blob/main/LSTM%26GRU_Keras_stockprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# LSTM or GRU implemented in Keras for Stock Prediction ( Time Series )



**Get NASDAQ data ( https://finance.yahoo.com/quote/GOOG/history/ ) loaded in a Pandas Dataframe**

In [ ]:
# import all libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# mount Google Drive to access Data
!fusermount -u drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
!ls "gdrive/My Drive/Colab Notebooks/"

# Get NASDAQ data ( https://finance.yahoo.com/quote/GOOG/history/ ) loaded in a Pandas Dataframe
stock_data = pd.read_csv("gdrive/My Drive/Colab Notebooks/GOOG.csv")
stock_data.info()

fusermount: failed to unmount /content/drive: No such file or directory
Mounted at /content/gdrive
_about.txt	   jet_images.h5	  ntuple_SingleMuon_Endcap_9.root
bolsas_astro.txt   mlp_vbfhzz		  ntuple_SingleNeutrino_PU200_63.root
bolsas_fisica.txt  model_best.pt	  pq_astronomia_2020.txt
events.root	   my_signal.csv	  pq_geral_2020.txt
GOOG.csv	   ntuple_bkg_ZZ4mu.root  pq_hep_2020.txt
graphs		   ntuple_ggH_ZZ4mu.root  RaioX_procedimento.pages
graphs.zip	   ntuple_qqH_ZZ4mu.root  VBFHZZ_signal.csv


FileNotFoundError: ignored

**We will take the average of the low and high of the Google stock for the day and volume of the stocks traded for the day to predict the stock prices.**

In [ ]:
import math

# take the average of the low and high of the Google stock for the day
stock_data["average"] = (stock_data["High"] + stock_data["Low"])/2
print(stock_data.head())

# take the volume and average stock price as our input features and store it in input_data list
input_feature= stock_data.iloc[:,[6,7]].values
input_data = input_feature
print("Input data shape:",input_data.shape)

# plot the data for volume for the Google stocks traded for the day
plt.plot(input_feature[:,0])
plt.title("Volume of stocks sold")
plt.xlabel("Time (latest-> oldest)")
plt.ylabel("Volume of stocks traded")
plt.show()


# plot the data for the average price for the day the Google stock
plt.plot(input_feature[:,1], color='blue')
plt.title("Google Stock Prices")
plt.xlabel("Time (latest-> oldest)")
plt.ylabel("Stock Opening Price")
plt.show()

# Normalizing the input data using MinMaxScaler so that all the input features are on the scale from 0 to 1
from sklearn.preprocessing import MinMaxScaler
sc= MinMaxScaler(feature_range=(0,1))
input_data[:,0:2] = sc.fit_transform(input_feature[:,:])



**The input to every LSTM layer must be in a 3D format. The three dimensions of this input are:**
*   **Samples:** one sequence is one sample and a batch is comprised of one or more samples
*   **Time Steps:** one time step is one point of observation in the sample
*   **Features:** one feature is one observation at a time step

**This means that the input layer expects a 3D array of data when fitting the model and when making predictions, even if specific dimensions of the array contain a single value ( e.g. one sample or one feature ). The input layer of your LSTM network assumes you have 1 or more samples and requires that you specify the number of time steps and the number of features. This is specifyied as a tuple to the input_shape argument.**




---


For more information see https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/




---





**We need to put the LSTM input data in the form (Volume of stocks traded, Average stock price). We also need to create a time series containing this data for past 50 days and the target variable that will be Google’s daily stock price.**

In [ ]:
# Rescale data to [0,1] interval to make it easier to train the NN
#from sklearn.preprocessing import MinMaxScaler
#sc = MinMaxScaler(feature_range = (0, 1))
#input_data = sc.fit_transform(input_data)

# Create the time series with lengh given by LOOKBACK
lookback= 30

test_size=int(.3 * len(stock_data))
X=[]
y=[]
for i in range(len(stock_data)-lookback-1):
    t=[]
    for j in range(0,lookback):        
        t.append(input_data[[(i+j)], :])
    X.append(t)
    y.append(input_data[i+ lookback,1])


**The input data 3D format is (sample size, time steps, no. of input features).  In our case timesteps is given by LOOKBACK and the number of input features is 2 (volume of stocks traded and the average stock price).**

In [ ]:
X, y= np.array(X), np.array(y)
#X_test = X[:test_size+lookback]
####### My mods to separate trainig from test data
#X_test = X[len(stock_data)-test_size-lookback:len(stock_data)]
X_test = X[len(stock_data)-lookback-test_size:len(stock_data)-lookback]
print("init=",len(stock_data)-lookback-test_size)
print("end=",len(stock_data)-lookback)
print("end=",len(stock_data))
X=X[0:len(stock_data)-lookback-test_size]
y=y[0:len(stock_data)-lookback-test_size]
#######
X = X.reshape(X.shape[0],lookback, 2)
X_test = X_test.reshape(X_test.shape[0],lookback, 2)

# Print dimension of the training data and test data: No. of sample, time steps and no. of input features
print("Total number of days:",len(stock_data))
print("X=",X.shape)
print("X_test=",X_test.shape)
print("y=",y.shape[0])

**Build the LSTM**

In [ ]:
from keras import Sequential
from keras.layers import Dense, LSTM , GRU

# define the LSTM architecture 
model = Sequential()
#model.add(LSTM(units=30, input_shape=(X.shape[1],2)))
model.add(GRU(units=30, input_shape=(X.shape[1],2)))
model.add(Dense(units=1,activation='linear'))
model.summary()

# compile 
#model.compile(optimizer='adam', loss='mean_squared_error', metrics=["mean_squared_error"])
model.compile(optimizer='adam', loss='mean_squared_error')
# train the network
history = model.fit(X, y, epochs=10, batch_size=32 , verbose=1 , validation_split=0.2)



**Make predictions and plot performance**

In [ ]:
# make predictions
Y_test = model.predict(X_test)

# Remove normalization by going back to original range of stock values
#Y_test=sc.inverse_transform(Y_test)
#input_data=sc.inverse_transform(input_data)

# plot the predictions
plt.plot(Y_test, color= 'red')
plt.plot(X_test[:,-1,1], color='green')
plt.title("Stock Prediction")
plt.xlabel("Time (latest-> oldest)")
plt.ylabel("Stock Average Price")
plt.show()

# plot performance
plt.style.use('default')
plt.figure(figsize=(10,7))

ax = plt.subplot(2, 2, 1)
ax.plot(history.history['loss'], label='loss')
ax.plot(history.history['val_loss'], label='val_loss')
ax.set_ylim([0, 0.1])
ax.legend(loc="upper right")
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')


